# Set up the environment

In [3]:
%%capture

import contextlib, io, warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
plt.plot([0,1],[1,0])
plt.show()
import hide_imports
from neuronunit.optimisation.optimization_management import inject_and_plot_model, OptMan
import copy
import pickle
from neuronunit.optimisation.optimization_management import check_match_front
from neuronunit.optimisation.optimization_management import check_binary_match



# Design simulated data tests

In [ ]:
backend = ""
MU = ""
NGEN = ""

In [ ]:

test_frame = pickle.load(open('processed_multicellular_constraints.p','rb'))
stds = {}
for k,v in hide_imports.TSD(test_frame['Neocortex pyramidal cell layer 5-6']).items():
    temp = hide_imports.TSD(test_frame['Neocortex pyramidal cell layer 5-6'])[k]
    stds[k] = temp.observation['std']
    #print((temp.name,temp.observation))
OMObjects = []
cloned_tests = copy.copy(test_frame['Neocortex pyramidal cell layer 5-6'])
if backend == "RAW":
    fp = ['a','b','c','d','k']
elif backend == "HH":
    fp = ['g_L',
        'g_K',
        'g_Na',
        'vr']


In [ ]:
OM = OptMan(hide_imports.TSD(cloned_tests),backend=backend)
OM.tests = hide_imports.TSD(cloned_tests)
simulated_data_tests, OM, target = OM.make_sim_data_tests(backend,MU,NGEN,free_parameters=fp)


# Show what the randomly generated target waveform the optimizer needs to find actually looks like

In [ ]:
target.rheobase
inject_and_plot_model(target)

# first lets just optimize over all objective functions all the time.
# Commence optimization of models on simulated data sets

In [ ]:
%%capture
s = io.StringIO()
with contextlib.redirect_stdout(s):
    ga_out = simulated_data_tests.optimize(backend=OM.backend,\
            protocol={'allen': False, 'elephant': True},\
                MU=MU,NGEN=NGEN, free_params=['a','b','c','d'])
    opt = ga_out['pf'][0].dtc
    opt = OM.get_agreement(opt)


In [ ]:
plt.rcParams.update({'font.size': 15})
front = ga_out['front']

check_match_front(target,front[0:10])



In [ ]:
check_binary_match(opt,target)

In [ ]:
opt.SA

In [ ]:
opt.agreement

# How about using all the objectives together?
